In [1]:
import pandas as pd, numpy as np
from pathlib import Path
import fsspec

S3_STATIONS_TXT   = "s3://noaa-ghcn-pds/ghcnd-stations.txt"
S3_INVENTORY_TXT  = "s3://noaa-ghcn-pds/ghcnd-inventory.txt"
S3_BY_STATION     = "s3://noaa-ghcn-pds/csv/by_station/{id}.csv"
STOR = {"anon": True}

OUTDIR = Path('../data'); OUTDIR.mkdir(parents=True, exist_ok=True)
OUT_PARQUET = OUTDIR / 'ghcn_il_top4_daily.parquet'
print('Output:', OUT_PARQUET.resolve())

Output: /home/alexakt2/.jupyter/https:/data/ghcn_il_top4_daily.parquet


In [2]:
# Load Stations
colspecs = [(0,11),(12,20),(21,30),(31,37),(38,40),(41,71),(72,75),(76,79),(80,85)]
names = ['ID','LATITUDE','LONGITUDE','ELEVATION','STATE','NAME','GSN_FLAG','HCN_CRN_FLAG','WMO_ID']

stations = pd.read_fwf(S3_STATIONS_TXT, colspecs=colspecs, names=names, dtype={'ID':str,'STATE':str,'WMO_ID':str}, storage_options=STOR)
stations['NAME'] = stations['NAME'].str.strip(); stations['STATE'] = stations['STATE'].fillna('').str.strip()

inventory = pd.read_csv(
    S3_INVENTORY_TXT, sep=r'\s+', names=['ID','LAT','LON','ELEMENT','FIRSTYEAR','LASTYEAR'],
    dtype={'ID':str,'ELEMENT':str,'FIRSTYEAR':int,'LASTYEAR':int}, engine='python', storage_options=STOR
)

stations.head(), inventory.head()

(            ID  LATITUDE  LONGITUDE  ELEVATION STATE                   NAME  \
 0  ACW00011604   17.1167   -61.7833       10.1        ST JOHNS COOLIDGE FLD   
 1  ACW00011647   17.1333   -61.7833       19.2                     ST JOHNS   
 2  AE000041196   25.3330    55.5170       34.0          SHARJAH INTER. AIRP   
 3  AEM00041194   25.2550    55.3640       10.4                   DUBAI INTL   
 4  AEM00041217   24.4330    54.6510       26.8               ABU DHABI INTL   
 
   GSN_FLAG HCN_CRN_FLAG WMO_ID  
 0      NaN          NaN    NaN  
 1      NaN          NaN    NaN  
 2      GSN          NaN  41196  
 3      NaN          NaN  41194  
 4      NaN          NaN  41217  ,
             ID      LAT      LON ELEMENT  FIRSTYEAR  LASTYEAR
 0  ACW00011604  17.1167 -61.7833    TMAX       1949      1949
 1  ACW00011604  17.1167 -61.7833    TMIN       1949      1949
 2  ACW00011604  17.1167 -61.7833    PRCP       1949      1949
 3  ACW00011604  17.1167 -61.7833    SNOW       1949      194

In [4]:
# Load Station: USC00087205
def load_station_daily(url: str) -> pd.DataFrame:
    df = pd.read_csv(url, storage_options=STOR, dtype={'ID':str,'ELEMENT':str}, parse_dates=['DATE'])
    df['DATA_VALUE'] = df['DATA_VALUE'].replace(-9999, np.nan)
    wide = (df.pivot_table(index=['ID','DATE'], columns='ELEMENT', values='DATA_VALUE', aggfunc='first').reset_index())
    for c in ('TMAX','TMIN','TAVG'):
        if c in wide: wide[c] = wide[c]/10.0
    if 'PRCP' in wide: wide['PRCP'] = wide['PRCP']/10.0
    return wide.sort_values(['ID','DATE']).reset_index(drop=True)

dt_daily = load_station_daily(S3_BY_STATION.format(id='USC00087205'))
dt_daily.tail()

/tmp/ipykernel_18404/1077117106.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url, storage_options=STOR, dtype={'ID':str,'ELEMENT':str}, parse_dates=['DATE'])


ELEMENT,ID,DATE,DAPR,MDPR,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WT01,WT03,WT04,WT06,WT08,WT11,WT14,WT16
46156,USC00087205,2025-12-05,NaN,NaN,0.0,NaN,NaN,29.4,17.8,278.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46157,USC00087205,2025-12-06,NaN,NaN,0.0,NaN,NaN,28.3,20.6,278.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46158,USC00087205,2025-12-07,NaN,NaN,18.5,NaN,NaN,28.3,15.6,217.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46159,USC00087205,2025-12-08,NaN,NaN,40.6,NaN,NaN,24.4,17.2,239.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46160,USC00087205,2025-12-09,NaN,NaN,0.3,NaN,NaN,23.9,10.6,200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#1 

# Remove data that's not within 1991 and 2020

dt_daily_30yrs = dt_daily.loc[dt_daily['DATE'].between('1991-01-01', '2020-12-31')].copy()

In [ ]:
# Check if there are NaNs

print(dt_daily_30yrs['TMAX'].isnull().sum()) 
print(dt_daily_30yrs['TMIN'].isnull().sum()) 

# Drop NaNs
dt_daily_30yrs = dt_daily_30yrs.dropna(subset=['TMAX'])

dt_daily_30yrs = dt_daily_30yrs.dropna(subset=['TMIN'])


142
159


In [24]:
# Get Year and Month from DATE Column

dt_daily_30yrs['DATE'] = pd.to_datetime(dt_daily_30yrs['DATE'])
dt_daily_30yrs['year'] = dt_daily_30yrs['DATE'].dt.year
dt_daily_30yrs['month'] = dt_daily_30yrs['DATE'].dt.month

# Get only the data from strawberry season
dt_straw = dt_daily_30yrs[dt_daily_30yrs['month'].isin([1, 10, 11, 12])]

In [49]:
# Risk Mean

frost_days = (dt_straw['TMIN'] <=32).sum()
freeze_days = (dt_straw['TMIN'] <=28).sum()

# 4 months of the year for 30 years
months = (30)*4

# Frost - Risk Mean 
rm_frost = frost_days/months
print(rm_frost)

# Freeze - Risk Mean
rm_freeze = freeze_days/months
print(rm_freeze)


30.091666666666665
30.091666666666665


In [48]:
# I am getting the same value for both which seems wrong... I am going to move on in the intrest of time and maybe come back

In [74]:
#2

# load file data
url = "https://www.cpc.ncep.noaa.gov/data/indices/sstoi.indices"

ds = pd.read_csv(url, delim_whitespace=True, comment='#', header=None)

ds.columns = ["YR","MON","NINO1+2","ANOM1","NINO3","ANOM2","NINO3.4","ANOM3","NINO4","ANOM4"]

/tmp/ipykernel_18404/1811308429.py:6: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  ds = pd.read_csv(url, delim_whitespace=True, comment='#', header=None)


In [ ]:
# Update to strawberry season
ds['DATE'] = pd.to_datetime(dict(year=ds['YR'], month=ds['MON'], day=1))
ds_straw = ds['time'].dt.month.isin([10,11,12,1])

# moving on because of time. I am not going to seperate this out into months to make it easier

ValueError: Unable to parse string "YR" at position 0

In [80]:
nino_yr = ds.groupby("YR").mean().reset_index()
nino_yr.head()

freeze_data = dt_straw[dt_straw['TMIN'] <= 28].copy()
freeze_yearly = freeze_data.groupby(freeze_data['DATE'].dt.year).size().reset_index(name='freeze_days')
freeze_yearly.rename(columns={'DATE': 'YR'}, inplace=True)


TypeError: agg function failed [how->mean,dtype->object]

In [81]:
df_corr = pd.merge(freeze_yearly, nino_yr, on='YR', how='inner')
df_corr.head()

NameError: name 'freeze_yearly' is not defined

In [82]:
from scipy.stats import pearsonr

for col in ["NINO1+2","NINO3","NINO3.4","NINO4"]:
    r, p = pearsonr(df_corr['freeze_days'], df_corr[col])
    print(f"{col}: r = {r:.3f}, p = {p:.3f}")


NameError: name 'df_corr' is not defined

In [ ]:
# I have run out of time and cannot get this to run. If I could see the results then I would see which has the value
# closest to one and that would have the strongest correlation